# Data Pre-Processing For The Combined TV-NN Unrolled Algorithm

Folder Of Sea Turtle Head Images: https://www.kaggle.com/datasets/wildlifedatasets/seaturtleidheads. Download this folder of sea turtle heads and the filepath of this folder will be used for the input_folder variable in the crop_resize_tvnn function

### Import All Necessary Libraries

In [ ]:
from PIL import Image
import os
import shutil
import cv2
import numpy as np

### Create Ground Truth Training Data By Resizing and Reshaping All Remaining Images

NOTE: Make sure the image used in the Chambolle-Pock (which will be used in the testing phase of this model) has been deleted from orignial Kaggle image folder before any pre-processing so that the testing image is not used in the training phase

NOTE: Make sure the correct filepaths have been added

In [ ]:
#Crop, resize and greyscale image to create clean training data
def crop_resize_tvnn(input_folder, output_folder, target_size, num_images):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    count = 0

    for filename in os.listdir(input_folder):
        if count >= num_images:
            break

        try:
            img_path = os.path.join(input_folder, filename)
            img = Image.open(img_path)

            size = min(img.size)
            left = (img.size[0] - size) // 2
            top = (img.size[1] - size) // 2
            right = left + size
            bottom = top + size
            img = img.crop((left, top, right, bottom))

            img = img.resize((target_size, target_size), Image.ANTIALIAS)

            grayscale_img = img.convert("L")

            output_filename = f"{count + 1:04d}.jpg"
            output_path = os.path.join(output_folder, output_filename)
            grayscale_img.save(output_path)

            count += 1
        except Exception as e:
            print(f"Error processing {filename}: {e}")

if __name__ == "__main__":
    input_folder = r"<filepath of Kaggle folder>" ###Location of the Kaggle downloaded image folder (after deleting ZvLoSlLFIz.jpeg)
    output_folder = r"Combined_TVNN_Unrolled_Algorithm\Pre_Processed_Images_TVNN\Clean_Training_Images" ###Location where the clean training data will be saved
    target_size = 64 ###Select target image size
    num_images = 1000 ### Number of training images

    crop_resize_tvnn(input_folder, output_folder, target_size, num_images)

### Add Gaussian Noise To Ground Truth Training Dataset To Create Noisy Images Training Dataset

NOTE: Make sure the correct filepaths have been added

In [ ]:
# add variable levels of Gaussian noise to clean training data to create noisy training data
def add_gaussian_noise_tvnn(image, std):
    noise = np.random.normal(0, std, size=image.shape)
    noisy_image = image + noise
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

def add_variable_noise_tvnn(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    sigmas = [3, 10, 25, 50, 100, 125, 150, 175, 200, 215]
    num_images = 1000
    images_per_sigma = num_images // len(sigmas)
    
    for i, sigma in enumerate(sigmas):
        for j in range(images_per_sigma):
            image_number = i * images_per_sigma + j + 1
            filename = f"{image_number:04d}.jpg"
            
            image_path = os.path.join(input_folder, filename)
            if os.path.exists(image_path):
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                noisy_image = add_gaussian_noise_tvnn(image, sigma)
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, noisy_image)
            else:
                pass

if __name__ == "__main__":
    input_folder_path = r"Combined_TVNN_Unrolled_Algorithm\Pre_Processed_Images_TVNN\Clean_Training_Images" ###Location of the clean training data
    output_folder_path = r"Combined_TVNN_Unrolled_Algorithm\Pre_Processed_Images_TVNN\Noisy_Training_Images" ###Location where the noisy training data will be saved
    
    add_variable_noise_tvnn(input_folder_path, output_folder_path)